In [1]:
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sp

Using backend: tensorflow.compat.v1

2023-05-15 15:50:51.761285: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
non-resource variables are not supported in the long term
Enable just-in-time compilation with XLA.



2023-05-15 15:51:10.946211: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-15 15:51:11.278353: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-15 15:51:11.279026: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
R = 2
n = 4
q = 1
epsilon = 1

In [3]:
num_dense_nodes = 50
num_dense_layers = 7

In [4]:
def pde(x, y):
    ai_zeros, _, _, _ = sp.ai_zeros(n)
    alpha_n = ai_zeros[-1]
    E = -alpha_n * np.power((q ** 2) * (epsilon ** 2) / 2, (1 / 3))
    
    dy_xx = dde.grad.hessian(y, x, i=0, j=0)
    
    return dy_xx + 2 * (E - q * epsilon * x) * y

In [5]:
def psi(x):
    ai_zeros, _, _, _ = sp.ai_zeros(n)
    alpha_n = ai_zeros[-1]
    E = -alpha_n * np.power((q ** 2) * (epsilon ** 2) / 2, (1 / 3))
    
    cube_root = np.power(2 * q * epsilon, (1 / 3))
    
    ai_value, _, _, _ = sp.airy(cube_root * (x - (E / (q * epsilon))))
    _, aip_value, _, _ = sp.airy(-cube_root * (E / (q * epsilon)))
    
    normalization_constant = np.sqrt(cube_root / (aip_value ** 2))
    
    return normalization_constant * ai_value

In [6]:
domain = dde.geometry.Interval(0, R)

In [7]:
def get_collocation_points(n):
    rs = []
    
    r_interval = R / (3 * n)
    
    for k in range(3 * n - 1):
        r = (k + 1) * r_interval
        rs.append(r)
    
    return np.array(rs).reshape((3 * n - 1, 1))

In [8]:
collocation_points = get_collocation_points(n)
collocation_values = psi(collocation_points)

collocation_values

array([[-0.22429754],
       [-0.38540924],
       [-0.44099962],
       [-0.37954696],
       [-0.22055194],
       [-0.00663928],
       [ 0.2091959 ],
       [ 0.37723824],
       [ 0.46211298],
       [ 0.44904503],
       [ 0.3446705 ]])

In [9]:
ic = dde.icbc.PointSetBC(collocation_points, collocation_values)

In [10]:
def boundary(x, on_boundary):
    return on_boundary and np.isclose(0, x[0])

def boundary_value(x):
    return 0

dirichlet_bc = dde.icbc.DirichletBC(domain, boundary_value, boundary)

In [11]:
bcs = [ic, dirichlet_bc]

In [12]:
num_train = 64
num_test = 100
num_boundary = 2

singularity = [R]

data = dde.data.PDE(
    domain, 
    pde, 
    bcs, 
    num_domain=num_train,
    num_boundary=num_boundary,
    solution=psi, 
    num_test=num_test,
    exclusions=singularity
)

In [13]:
activation = 'tanh'
net = dde.nn.FNN(
    [1] + [num_dense_nodes] * num_dense_layers + [1], 
    activation, 
    'Glorot uniform'
)

In [14]:
loss_weights = [1, 100, 100]

model = dde.Model(data, net)
model.compile(
    'adam', 
    lr=0.0001, 
    metrics=['l2 relative error'], 
    loss_weights=loss_weights
)

Compiling model...
Building feed-forward neural network...
'build' took 1.190798 s



/home/piotr/repos/quant-chem-pinns/venv/lib/python3.10/site-packages/deepxde/nn/tensorflow_compat_v1/fnn.py:114: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(
2023-05-15 15:53:34.399055: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-15 15:53:34.399326: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-15 15:53:34.399506: I tensorflow/compiler/x

'compile' took 8.875186 s



In [15]:
loss_history, train_state = model.train(iterations=20000)

Initializing variables...
Training model...



2023-05-15 15:53:49.245546: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2023-05-15 15:53:49.432848: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7fc750005f40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-15 15:53:49.432882: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce 940MX, Compute Capability 5.0
2023-05-15 15:53:49.442036: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-15 15:53:51.481486: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8901
2023-05-15 15:53:58.487611: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Step      Train loss                        Test loss                         Test metric   
0         [6.57e+00, 3.18e+01, 0.00e+00]    [6.77e+00, 3.18e+01, 0.00e+00]    [1.67e+00]    
1000      [7.22e-01, 2.99e-01, 7.86e-05]    [7.35e-01, 2.99e-01, 7.86e-05]    [1.69e-01]    
2000      [4.59e-03, 1.97e-03, 1.42e-05]    [4.32e-03, 1.97e-03, 1.42e-05]    [1.33e-02]    
3000      [7.40e-04, 7.55e-06, 8.20e-08]    [7.57e-04, 7.55e-06, 8.20e-08]    [7.90e-04]    
4000      [4.57e-04, 2.62e-06, 1.20e-08]    [4.84e-04, 2.62e-06, 1.20e-08]    [4.74e-04]    
5000      [3.75e-04, 5.40e-06, 3.48e-08]    [3.97e-04, 5.40e-06, 3.48e-08]    [7.29e-04]    
6000      [3.49e-04, 4.06e-05, 2.06e-07]    [3.70e-04, 4.06e-05, 2.06e-07]    [1.86e-03]    
7000      [9.39e-04, 1.12e-03, 3.57e-06]    [9.56e-04, 1.12e-03, 3.57e-06]    [1.00e-02]    
8000      [2.72e-04, 7.91e-07, 2.02e-09]    [2.80e-04, 7.91e-07, 2.02e-09]    [2.85e-04]    
9000      [2.50e-04, 1.04e-06, 1.45e-08]    [2.54e-04, 1.04e-06, 1.45e

In [ ]:
dde.saveplot(loss_history, train_state, issave=True, isplot=True)